# CS483 - Colab 2
## Frequent Pattern Mining in Spark

In [5]:
#This code is written by Parikha Goyanka.
#Refrences used (mainly): https://engineering.empathy.co/frequent-pattern-mining/, Google search to undersatnd and deal with errors.
from IPython.display import Image
print("Colab 2 Mascot")
Image(url='https://cdn.dribbble.com/users/222579/screenshots/1654898/stubby-ben-rex-roll.gif',width=150)

Colab 2 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [6]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=1ee556ff3c9856131a710201b4c681e38cb19097b77b336a306b5a1d23764067
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [11]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [12]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [13]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [14]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [15]:
''' 2 lines of code expected '''
transc_ = orders.join(products, on='product_id').groupby('order_id').agg(collect_list('product_name').alias('products'))
transc_.take(2)


[Row(order_id=1, products=['Bulgarian Yogurt', 'Organic 4% Milk Fat Whole Milk Cottage Cheese', 'Organic Celery Hearts', 'Cucumber Kirby', 'Lightly Smoked Sardines in Olive Oil', 'Bag of Organic Bananas', 'Organic Hass Avocado', 'Organic Whole String Cheese']),
 Row(order_id=96, products=['Roasted Turkey', 'Organic Cucumber', 'Organic Grape Tomatoes', 'Organic Pomegranate Kernels', 'Organic Raspberries', 'Organic Whole Strawberries', 'Organic Blueberries'])]

In [16]:
# Check the Schema to understand data
transc_.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- products: array (nullable = false)
 |    |-- element: string (containsNull = false)



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [17]:
''' 3 lines of code expected '''
# Importing the Frequent pattern minning library
from pyspark.ml.fpm import FPGrowth

my_fpGrowth_ = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.5)
# Creating the FPM model
my_model = my_fpGrowth_.fit(transc_)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [18]:
my_model.freqItemsets.show()
my_model.associationRules.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|      [Green Onions]| 1445|
|   [Red Raspberries]| 1493|
|    [Organic Banana]| 2332|
|  [Jalapeno Peppers]| 1899|
|[Organic Large Ex...| 2891|
|[Organic Whole St...| 1993|
|[Organic Peeled W...| 2460|
|             [Limes]| 6033|
|[Limes, Large Lemon]| 1595|
|     [Limes, Banana]| 1331|
|       [Raspberries]| 3279|
|      [Hass Avocado]| 1633|
|[Organic Broccoli...| 1361|
|[Uncured Genoa Sa...| 1788|
|      [Spring Water]| 2225|
|[Michigan Organic...| 2627|
|     [Yellow Onions]| 3762|
|[Organic Strawber...|10894|
|[Organic Strawber...| 3074|
|[Organic Strawber...| 2174|
+--------------------+-----+
only showing top 20 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [19]:
my_fpGrowth_ = FPGrowth(itemsCol="products", minSupport=0.001, minConfidence=0.5)
my_model = my_fpGrowth_.fit(transc_)
my_model.freqItemsets.show()
my_model.associationRules.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[White Cheddar Po...| 370|
|[Organic YoKids V...| 259|
|[Total 0% Nonfat ...| 993|
|[Total 0% Nonfat ...| 137|
|[Total 0% Nonfat ...| 157|
|[Total 0% Nonfat ...| 143|
|[Total 0% Nonfat ...| 258|
|[Organic Large Gr...| 730|
|[Organic Large Gr...| 185|
|[Organic Large Gr...| 263|
|[Organic Large Gr...| 136|
|[Organic Large Gr...| 164|
|[Organic Cream Ch...| 365|
|[Organic Reduced ...| 140|
|[Florida Orange J...| 283|
|      [Green Onions]|1445|
|[Green Onions, Li...| 206|
|[Green Onions, Or...| 133|
|[Green Onions, Bu...| 187|
|[Green Onions, La...| 229|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+---------------

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [20]:
# stoping Spark Environment
sc.stop()

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!